In [1]:
import sys
import os
import math
import moveit_commander
import moveit_msgs.msg
import rospy
import numpy as np
import trajectory_msgs.msg
from geometry_msgs.msg import PoseStamped
import tf
import actionlib
### ALL OF THESE IMPORTS MUST BE HANDLeD BY TAKESHI INSTALL

the rosdep view is empty: call 'sudo rosdep init' and 'rosdep update'


In [2]:
rospy.init_node('moveiteando')

In [3]:
listener = tf.TransformListener()

In [4]:
arm = moveit_commander.MoveGroupCommander('arm')
base=  moveit_commander.MoveGroupCommander('base')
gripper = moveit_commander.MoveGroupCommander('gripper')
head = moveit_commander.MoveGroupCommander('head')

In [5]:
z= 0 ## QUERY Z
(trans,rot) = listener.lookupTransform('/map', '/head_rgbd_sensor_rgb_frame', rospy.Time(0))
e = np.rad2deg(tf.transformations.euler_from_quaternion(rot))
D_map=np.abs(trans[2]-z)/np.tan(np.deg2rad(e[0]-(-90)))
print ('x,y',D_map*-1* np.sin(np.deg2rad(e[2])), D_map* np.cos(np.deg2rad(e[2]))  )



('x,y', 1089.8550835887186, -1.2750910684917274)


In [6]:
#####SPAWN OBJECTS
from gazebo_ros import gazebo_interface
from geometry_msgs.msg import Pose, Quaternion
def spawn_object(gazebo_name, name, x, y, z, yaw):
    rospy.loginfo('Spawn: {0}'.format(name))
    initial_pose = Pose()
    initial_pose.position.x = x
    initial_pose.position.y = y
    initial_pose.position.z = z
    roll = 0.0
    pitch = 0.0
    yaw = 0.0
    q = tf.transformations.quaternion_from_euler(roll, pitch, yaw)
    initial_pose.orientation = Quaternion(q[0], q[1], q[2], q[3])

    path_xml = _path_xml.replace('MODEL_NAME', name)

    with open(path_xml, "r") as f:
        model_xml = f.read()

    model_xml = model_xml.replace('PATH_TO_MODEL', _path_model)

    gazebo_interface.spawn_sdf_model_client(gazebo_name, model_xml, rospy.get_namespace(),
                                            initial_pose, "", "/gazebo")


In [7]:
_path_xml = "/home/roboworks/Codes/ycb_ws/src/robot_object_views/robotobject/models/MODEL_NAME/model-1_4.sdf"
_path_model = "/home/roboworks/Codes/ycb_ws/src/robot_object_views/robotobject/models"

model_database_template = """<sdf version="1.4">
  <world name="default">
    <include>
      <uri>model://MODEL_NAME</uri>
    </include>
  </world>
</sdf>"""

objs=os.listdir(_path_model)

In [8]:
model_name= "073-g_lego_duplo"
spawn_object("073-g_lego_duplo", "073-g_lego_duplo", -1.5, 0.5, 1, 0)


[INFO] [1609433573.613004, 345.538000]: Spawn: 073-g_lego_duplo
[INFO] [1609433573.615414, 345.538000]: Waiting for service /gazebo/spawn_sdf_model
[INFO] [1609433573.628981, 345.544000]: Calling service /gazebo/spawn_sdf_model
[INFO] [1609433573.642708, 345.547000]: Spawn status: SpawnModel: Failure - entity already exists.


In [9]:
goal=base.get_current_joint_values()
goal[0]=1
goal


[1, -0.00013686033108162583, -0.0011686598826976839]

In [12]:
base.set_joint_value_target(goal)
base.go()

True

[1, -0.00013641330278248986, -0.001166446439266248]

In [24]:
pp=whole_body.get_current_joint_values()

NameError: name 'whole_body' is not defined